<a href="https://colab.research.google.com/github/KirPim/DS_school_2020_home_work/blob/main/HW4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1. Запустим database session

In [312]:
from sqlalchemy import create_engine
#engine = create_engine('sqlite:///example.db', echo=True)
engine = create_engine('sqlite:///:memory:', echo=True)
#engine = create_engine('sqlite:///:memory:')
conn = engine.connect()

from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

2020-12-14 19:15:31,001 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-12-14 19:15:31,003 INFO sqlalchemy.engine.base.Engine ()
2020-12-14 19:15:31,005 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-12-14 19:15:31,007 INFO sqlalchemy.engine.base.Engine ()


### 2. Вспомогательные функции для печати и вывода результатов SQL запросов

In [313]:
from IPython.display import display
import pandas as pd
import sqlalchemy

def sql(query):
    print()
    print(query)
    print()

def get_results(query):
    global engine
    q = query.statement if isinstance(query, sqlalchemy.orm.query.Query) else query
    return pd.read_sql(q, engine)

def display_results(query):
    df = get_results(query)
    display(df)
    #sql(query)

### 3. Инициализация схемы БД

Схема - это пространство имен, которое содержит именованные объекты базы данных, такие как таблицы, представления, индексы, типы данных

In [314]:
!pip install sqlalchemy_explore

In [315]:
from sqlalchemy.ext.declarative import declarative_base
import sqlalchemy_explore

### the basic base class for SQLAlchemy schema objects
# Base = declarative_base(bind=engine)

### base class including utils like an __repr__ method
### see https://pypi.org/project/sqlalchemy-explore/
Base = declarative_base(cls=sqlalchemy_explore.ReflectiveMixin)

### Создание самой схемы

In [316]:
from sqlalchemy import Column, DateTime, ForeignKey, Integer, NVARCHAR, Numeric, Sequence
from sqlalchemy.orm import relationship

class Customer(Base):
    __tablename__ = 'customers'

    CustomerId = Column(Integer, Sequence('customer_id_seq'), primary_key=True)
    FirstName = Column(NVARCHAR(40), nullable=False)
    LastName = Column(NVARCHAR(20), nullable=False)
    Company = Column(NVARCHAR(80))
    Address = Column(NVARCHAR(70))
    Phone = Column(NVARCHAR(24))
    Email = Column(NVARCHAR(60), nullable=False)
    
class Item(Base):
    __tablename__ = 'items'
    
    ItemId = Column(Integer, Sequence('item_id_seq'), primary_key=True)
    Name = Column(NVARCHAR(40), nullable=False)
    Price = Column(Numeric, nullable=False)

class Purchase(Base):
    __tablename__ = 'purchases'
    
    PurchaseId = Column(Integer, Sequence('purchase_id_seq'), primary_key=True)
    ItemId = Column(ForeignKey('items.ItemId'), nullable=False, index=True)
    CustomerId = Column(ForeignKey('customers.CustomerId'), nullable=False, index=True)
    Date = Column(DateTime, nullable=False)
    
    item = relationship('Item')
    customer = relationship('Customer')

In [317]:
Purchase.ItemId.name

'ItemId'

In [318]:
Purchase.CustomerId.name

'CustomerId'

In [319]:
Purchase.PurchaseId.name

'PurchaseId'

### 5. Создадим таблицы в базе данных в соответствии со схемой

In [320]:
Base.metadata.create_all(engine)

2020-12-14 19:15:34,751 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("customers")
2020-12-14 19:15:34,753 INFO sqlalchemy.engine.base.Engine ()
2020-12-14 19:15:34,756 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("customers")
2020-12-14 19:15:34,758 INFO sqlalchemy.engine.base.Engine ()
2020-12-14 19:15:34,761 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("items")
2020-12-14 19:15:34,762 INFO sqlalchemy.engine.base.Engine ()
2020-12-14 19:15:34,763 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("items")
2020-12-14 19:15:34,764 INFO sqlalchemy.engine.base.Engine ()
2020-12-14 19:15:34,765 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("purchases")
2020-12-14 19:15:34,767 INFO sqlalchemy.engine.base.Engine ()
2020-12-14 19:15:34,769 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("purchases")
2020-12-14 19:15:34,769 INFO sqlalchemy.engine.base.Engine ()
2020-12-14 19:15:34,771 INFO sqlalchemy.engine.base.Engine 
CREATE

In [321]:
engine.table_names()

2020-12-14 19:15:34,793 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2020-12-14 19:15:34,794 INFO sqlalchemy.engine.base.Engine ()


['customers', 'items', 'purchases']

### 6. Создадим покупателя

In [322]:
moshe = Customer(
    FirstName='Moshe', 
    LastName='Cohen', 
    Address='Alenbi 99, Tel Aviv', 
    Phone="053-5556789", 
    Email='moshe@cohen.com')

session.add(moshe)
session.commit()

2020-12-14 19:15:34,824 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-14 19:15:34,829 INFO sqlalchemy.engine.base.Engine INSERT INTO customers ("FirstName", "LastName", "Company", "Address", "Phone", "Email") VALUES (?, ?, ?, ?, ?, ?)
2020-12-14 19:15:34,830 INFO sqlalchemy.engine.base.Engine ('Moshe', 'Cohen', None, 'Alenbi 99, Tel Aviv', '053-5556789', 'moshe@cohen.com')
2020-12-14 19:15:34,832 INFO sqlalchemy.engine.base.Engine COMMIT


### 7. Выполним запрос

Используя язык выражений SQLAchemy

In [323]:
from sqlalchemy import select 

customers_query = select([Customer.FirstName, Customer.LastName, Customer.Email])
results = conn.execute(customers_query)

print()
for row in results:
    print(row)

print()
print(type(row)) # rows are of type sqlalchemy.engine.result.RowProxy

2020-12-14 19:15:34,846 INFO sqlalchemy.engine.base.Engine SELECT customers."FirstName", customers."LastName", customers."Email" 
FROM customers
2020-12-14 19:15:34,848 INFO sqlalchemy.engine.base.Engine ()

('Moshe', 'Cohen', 'moshe@cohen.com')

<class 'sqlalchemy.engine.result.RowProxy'>


In [324]:
display_results(customers_query)

2020-12-14 19:15:34,865 INFO sqlalchemy.engine.base.OptionEngine SELECT customers."FirstName", customers."LastName", customers."Email" 
FROM customers
2020-12-14 19:15:34,867 INFO sqlalchemy.engine.base.OptionEngine ()


,FirstName,LastName,Email
0,Moshe,Cohen,moshe@cohen.com


### 8. Добавим еще покупателей

In [325]:
Lisa = Customer(
    FirstName='Lisa',
    LastName='Cohen', 
    Address='Alenbi 99, Tel Aviv', 
    Phone="052-1234565", 
    Email='lisa@cohen.com')

session.add(Lisa)
session.commit()

2020-12-14 19:15:34,900 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-14 19:15:34,902 INFO sqlalchemy.engine.base.Engine INSERT INTO customers ("FirstName", "LastName", "Company", "Address", "Phone", "Email") VALUES (?, ?, ?, ?, ?, ?)
2020-12-14 19:15:34,903 INFO sqlalchemy.engine.base.Engine ('Lisa', 'Cohen', None, 'Alenbi 99, Tel Aviv', '052-1234565', 'lisa@cohen.com')
2020-12-14 19:15:34,905 INFO sqlalchemy.engine.base.Engine COMMIT


In [326]:
Nika = Customer(
    FirstName='Nika', 
    LastName='Rave', 
    Address='Green st, LA', 
    Phone="330-1234565", 
    Email='Nika@rave.com')

session.add(Nika)
session.commit()

2020-12-14 19:15:34,917 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-14 19:15:34,920 INFO sqlalchemy.engine.base.Engine INSERT INTO customers ("FirstName", "LastName", "Company", "Address", "Phone", "Email") VALUES (?, ?, ?, ?, ?, ?)
2020-12-14 19:15:34,921 INFO sqlalchemy.engine.base.Engine ('Nika', 'Rave', None, 'Green st, LA', '330-1234565', 'Nika@rave.com')
2020-12-14 19:15:34,924 INFO sqlalchemy.engine.base.Engine COMMIT


In [327]:
Lisa_2 = Customer(
    FirstName='Lisa',
    LastName='White', 
    Address='Alenbi 66, Tel Aviv', 
    Phone="062-1234565", 
    Email='lisa@White.com')

session.add(Lisa_2)
session.commit()

2020-12-14 19:15:34,937 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-14 19:15:34,939 INFO sqlalchemy.engine.base.Engine INSERT INTO customers ("FirstName", "LastName", "Company", "Address", "Phone", "Email") VALUES (?, ?, ?, ?, ?, ?)
2020-12-14 19:15:34,940 INFO sqlalchemy.engine.base.Engine ('Lisa', 'White', None, 'Alenbi 66, Tel Aviv', '062-1234565', 'lisa@White.com')
2020-12-14 19:15:34,942 INFO sqlalchemy.engine.base.Engine COMMIT


# ДЗ
 Дедлайн **15.12.2020, 23:59**
  
- Решить первые 20 задач с сайта https://www.sql-ex.ru (Раздел SELECT(обучающий этап)). Оценивается пропорционально кол-ву выполненных задач. + 20 % (еще 15 задач)

- Дополнить таблицу 'items' и 'purchases' 5ью экземплярами (каждую) на Ваш Выбор. Важно: они должны быть привязаны к существующим покупателям Lisa, Nika, Moshe.
- При помощи запроса SQLAlchemy вывести все покупки Lisa. Вывести тоже самое при помощи Pandas.


<center><img src='https://github.com/ddvika/Data-Science-School-2020/blob/main/lecture_5/imgs/i-know-sql.jpg?raw=true'></center>

In [328]:
#Дополнить таблицу 'items' и 'purchases' 5ью экземплярами (каждую) на Ваш Выбор. Важно: они должны быть привязаны к существующим покупателям Lisa, Nika, Moshe.

Item1 = Item(Name = 'apple', Price = 100)
Item2 = Item(Name = 'pear', Price = 150)
Item3 = Item(Name = 'lemon', Price = 300)
Item4 = Item(Name = 'orange', Price = 230)
Item5 = Item(Name = 'banana', Price = 120)

session.add(Item1)
session.add(Item2)
session.add(Item3)
session.add(Item4)
session.add(Item5)
session.commit()

2020-12-14 19:15:34,963 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-14 19:15:34,967 INFO sqlalchemy.engine.base.Engine INSERT INTO items ("Name", "Price") VALUES (?, ?)
2020-12-14 19:15:34,969 INFO sqlalchemy.engine.base.Engine ('apple', 100.0)
2020-12-14 19:15:34,971 INFO sqlalchemy.engine.base.Engine INSERT INTO items ("Name", "Price") VALUES (?, ?)
2020-12-14 19:15:34,972 INFO sqlalchemy.engine.base.Engine ('pear', 150.0)
2020-12-14 19:15:34,973 INFO sqlalchemy.engine.base.Engine INSERT INTO items ("Name", "Price") VALUES (?, ?)
2020-12-14 19:15:34,974 INFO sqlalchemy.engine.base.Engine ('lemon', 300.0)
2020-12-14 19:15:34,976 INFO sqlalchemy.engine.base.Engine INSERT INTO items ("Name", "Price") VALUES (?, ?)
2020-12-14 19:15:34,979 INFO sqlalchemy.engine.base.Engine ('orange', 230.0)
2020-12-14 19:15:34,980 INFO sqlalchemy.engine.base.Engine INSERT INTO items ("Name", "Price") VALUES (?, ?)
2020-12-14 19:15:34,981 INFO sqlalchemy.engine.base.Engine ('banana', 120.0

In [329]:
items_query = select([Item.Name, Item.Price])
display_results(items_query)

2020-12-14 19:15:34,993 INFO sqlalchemy.engine.base.OptionEngine SELECT items."Name", items."Price" 
FROM items
2020-12-14 19:15:34,996 INFO sqlalchemy.engine.base.OptionEngine ()


/usr/local/lib/python3.6/dist-packages/sqlalchemy/sql/sqltypes.py:668: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  "storage." % (dialect.name, dialect.driver)


,Name,Price
0,apple,100.0
1,pear,150.0
2,lemon,300.0
3,orange,230.0
4,banana,120.0


In [330]:
display_results(customers_query)

2020-12-14 19:15:35,029 INFO sqlalchemy.engine.base.OptionEngine SELECT customers."FirstName", customers."LastName", customers."Email" 
FROM customers
2020-12-14 19:15:35,032 INFO sqlalchemy.engine.base.OptionEngine ()


,FirstName,LastName,Email
0,Moshe,Cohen,moshe@cohen.com
1,Lisa,Cohen,lisa@cohen.com
2,Nika,Rave,Nika@rave.com
3,Lisa,White,lisa@White.com


In [331]:
import datetime
Purchase1 = Purchase(ItemId = 1, CustomerId = 2, Date = datetime.date(2020, 12, 14))
Purchase2 = Purchase(ItemId = 3, CustomerId = 4, Date = datetime.date(2020, 12, 10))
Purchase3 = Purchase(ItemId = 4, CustomerId = 1, Date = datetime.date(2020, 12, 12))
Purchase4 = Purchase(ItemId = 1, CustomerId = 3, Date = datetime.date(2020, 12, 5))
Purchase5 = Purchase(ItemId = 5, CustomerId = 2, Date = datetime.date(2020, 12, 14))

session.add(Purchase1)
session.add(Purchase2)
session.add(Purchase3)
session.add(Purchase4)
session.add(Purchase5)
session.commit()

2020-12-14 19:15:35,077 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-14 19:15:35,081 INFO sqlalchemy.engine.base.Engine INSERT INTO purchases ("ItemId", "CustomerId", "Date") VALUES (?, ?, ?)
2020-12-14 19:15:35,083 INFO sqlalchemy.engine.base.Engine (1, 2, '2020-12-14 00:00:00.000000')
2020-12-14 19:15:35,086 INFO sqlalchemy.engine.base.Engine INSERT INTO purchases ("ItemId", "CustomerId", "Date") VALUES (?, ?, ?)
2020-12-14 19:15:35,087 INFO sqlalchemy.engine.base.Engine (3, 4, '2020-12-10 00:00:00.000000')
2020-12-14 19:15:35,089 INFO sqlalchemy.engine.base.Engine INSERT INTO purchases ("ItemId", "CustomerId", "Date") VALUES (?, ?, ?)
2020-12-14 19:15:35,090 INFO sqlalchemy.engine.base.Engine (4, 1, '2020-12-12 00:00:00.000000')
2020-12-14 19:15:35,094 INFO sqlalchemy.engine.base.Engine INSERT INTO purchases ("ItemId", "CustomerId", "Date") VALUES (?, ?, ?)
2020-12-14 19:15:35,096 INFO sqlalchemy.engine.base.Engine (1, 3, '2020-12-05 00:00:00.000000')
2020-12-14 19:15

In [332]:
purchase_query = select([Purchase.ItemId, Purchase.CustomerId, Purchase.Date])
display_results(purchase_query)

2020-12-14 19:15:35,113 INFO sqlalchemy.engine.base.OptionEngine SELECT purchases."ItemId", purchases."CustomerId", purchases."Date" 
FROM purchases
2020-12-14 19:15:35,116 INFO sqlalchemy.engine.base.OptionEngine ()


,ItemId,CustomerId,Date
0,1,2,2020-12-14
1,3,4,2020-12-10
2,4,1,2020-12-12
3,1,3,2020-12-05
4,5,2,2020-12-14


In [333]:
#При помощи запроса SQLAlchemy вывести все покупки Lisa. Вывести тоже самое при помощи Pandas.

from sqlalchemy import select
sql_query = select([Customer.FirstName, Customer.LastName, Item.Name]).where((Customer.CustomerId == Purchase.CustomerId) & (Item.ItemId == Purchase.ItemId) & (Customer.FirstName == 'Lisa'))

# result = conn.execute(sql_query)
# for row in result:
#   print(row)

display_results(sql_query)

2020-12-14 19:15:35,154 INFO sqlalchemy.engine.base.OptionEngine SELECT customers."FirstName", customers."LastName", items."Name" 
FROM customers, items, purchases 
WHERE customers."CustomerId" = purchases."CustomerId" AND items."ItemId" = purchases."ItemId" AND customers."FirstName" = ?
2020-12-14 19:15:35,154 INFO sqlalchemy.engine.base.OptionEngine ('Lisa',)


,FirstName,LastName,Name
0,Lisa,Cohen,apple
1,Lisa,White,lemon
2,Lisa,Cohen,banana


In [334]:
sql_query = '''
select it.Name
      ,c.FirstName      
      ,c.LastName 
 from customers as c
 inner join purchases as p
  on c.CustomerId = p.CustomerId and c.FirstName = 'Lisa'
 inner join items as it
  on p.ItemId = it.ItemId  
'''

pd.read_sql(sql_query, conn)

2020-12-14 19:15:35,180 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("
select it.Name
      ,c.FirstName      
      ,c.LastName 
 from customers as c
 inner join purchases as p
  on c.CustomerId = p.CustomerId and c.FirstName = 'Lisa'
 inner join items as it
  on p.ItemId = it.ItemId  
")
2020-12-14 19:15:35,181 INFO sqlalchemy.engine.base.Engine ()
2020-12-14 19:15:35,183 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("
select it.Name
      ,c.FirstName      
      ,c.LastName 
 from customers as c
 inner join purchases as p
  on c.CustomerId = p.CustomerId and c.FirstName = 'Lisa'
 inner join items as it
  on p.ItemId = it.ItemId  
")
2020-12-14 19:15:35,183 INFO sqlalchemy.engine.base.Engine ()
2020-12-14 19:15:35,184 INFO sqlalchemy.engine.base.Engine 
select it.Name
      ,c.FirstName      
      ,c.LastName 
 from customers as c
 inner join purchases as p
  on c.CustomerId = p.CustomerId and c.FirstName = 'Lisa'
 inner join items as it
  on p.ItemId = it.

,Name,FirstName,LastName
0,apple,Lisa,Cohen
1,lemon,Lisa,White
2,banana,Lisa,Cohen
